In [29]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output, State, Event
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

ImportError: cannot import name 'Event' from 'dash.dependencies' (/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/dash/dependencies.py)

In [4]:
import pymongo

In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")

In [6]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client["1-grams"]

In [7]:
def load(word):
    try:
        df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
        df = df.sort_values(by=['time'])
        df['year'] = [date.year for date in df['time']]
        df['day'] = [date.timetuple().tm_yday for date in df['time']]
        return df
    except:
        print(str("No timeseries found for "+word))
        df = pd.DataFrame()
        return df

In [8]:
q = load('easter')
q.sort_values(by='rank')
q['rank'].max()

70064

# Input box for query

In [73]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}}),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    values = input_value.split(',')
    for item in values:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)
10.245.186.62 - - [18/Apr/2019 16:53:54] "GET / HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 16:53:54] "GET /styles.css HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 16:53:54] "GET /_dash-layout HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 16:53:54] "GET /_dash-dependencies HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 16:53:54] "POST /_dash-update-component HTTP/1.1" 200 -


# Dropdown for query

### Load wordlist

In [9]:
import csv
wordlist = []
with open('wordlist_from-df.csv', 'rt') as f:
    reader = csv.reader(f)
    next(reader)
    wordlist=list(reader)

In [10]:
wordDF=pd.DataFrame(wordlist, columns=['word','rank'])

In [11]:
wordDF=wordDF[wordDF['rank']!='Rank']

In [12]:
wordDF=wordDF[wordDF['rank']!='minRank']

In [13]:
wordDF['rank']=wordDF['rank'].astype(int);

### Dropdown with under 5k

In [22]:
wordDF[wordDF['rank']<1000].shape

(1056, 2)

In [14]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<1000]
for word in wordDF_filtered['word']:
    options.append({'label': word,'value': word})

In [16]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('This shows any one-grams that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)


### Dropdown of 1k with ability to add options

In [ ]:
options = []
wordDF_filtered = wordDF[wordDF['rank']<1000]
for word in wordDF_filtered['word']:
    options.append({'label': word,'value': word})

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 1k rank at any time (1,056 onegrams)'),
    html.Div([dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    html.P('Add one-grams to the list containing a custom string:'),
    dcc.Input(id='useradd', value=''),
    html.Button(id='submit', children='Submit'),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-dropdown', 'options'),[Input('submit', 'n_clicks')],[State('useradd', 'value')])
def callback(n_clicks, item):
    if len(item) < 5:
        wordDF_filtered = wordDF[wordDF['rank']<1000]
    else:
        wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
    for word in wordDF_filtered:
        options.append({'label': word[0],'value': word[0]})
    return options

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0',debug=True)

### Dynamic Dropdown (abandoned)

In [21]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.P('Select a one-gram that cracked the top 5k rank at any time (47,844 onegrams)'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','@bts_twt'], multi=True)]),
    dcc.Graph(id='my-graph',figure={'layout': {'yaxis': {'autorange': 'reversed', 'type':'log'}}})
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure

@app.callback(Output('my-dropdown', 'options'), [Input('my-dropdown', 'value')])

def update_options(input_value):
    for item in input_value:
        if len(item) < 5:
            wordDF_filtered = wordDF[wordDF['rank']<1000]
        else:
            wordDF_filtered = wordDF[wordDF['word'].str.contains(item)]
        for word in wordDF_filtered:
            options.append({'label': word[0],'value': word[0]})
    return options


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0',debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)
10.245.186.62 - - [22/Apr/2019 16:20:23] "GET / HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:20:23] "GET /styles.css HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:20:24] "GET /_dash-layout HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:20:24] "GET /_dash-dependencies HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:21:35] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:22:18] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.186.62 - - [22/Apr/2019 16:22:20] "POST /_dash-update-component HTTP/1.1" 200 -
